# N-MTLR + 5-Model Ensemble (Independent Seeds & Hyperparameters)

- Model: N-MTLR (torchmtlr)
- Ensemble: 5 models, each with its own Optuna optimization (different seeds)
- Metric: IPCW C-index (sksurv)
- Validation: nested CV (outer for evaluation, inner for hyperparameter tuning)
- Ensembling: rank-based aggregation of risk scores
- Statistical evaluation: mean, std, 95% CI, paired t-test (ensemble vs single model)

In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from torchmtlr import MTLR, mtlr_neg_log_likelihood
from torchmtlr.utils import encode_survival, make_time_bins

from sksurv.util import Surv
from sksurv.metrics import concordance_index_ipcw

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

import optuna
import matplotlib.pyplot as plt
from scipy import stats

# Global seeds
torch.manual_seed(42)
np.random.seed(42)

# Main configuration
N_MODELS_ENSEMBLE = 5        # number of models in the ensemble
N_SPLITS_OUTER = 5           # number of outer folds
N_SPLITS_INNER = 3           # number of inner folds (Optuna CV)
N_TRIALS_INNER = 30          # number of Optuna trials per model per outer fold
TAU_CINDEX = 7.0             # horizon for IPCW C-index

# 1. Data Loading & Preprocessing

In [27]:
df_train = pd.read_csv("../../data/train_enhanced.csv")
df_eval = pd.read_csv("../../data/eval_enhanced.csv")

print(f"Train data: {df_train.shape}")
print(f"Eval data : {df_eval.shape}")
print("\nTrain columns:")
print(df_train.columns.tolist())

Train data: (3173, 237)
Eval data : (1193, 235)

Train columns:
['ANC', 'ANC_missing', 'BM_BLAST', 'BM_BLAST_missing', 'CHR_12_count', 'CHR_17_count', 'CHR_1_count', 'CHR_20_count', 'CHR_21_count', 'CHR_2_count', 'CHR_4_count', 'CHR_5_count', 'CHR_7_count', 'CHR_X_count', 'CHR_nunique', 'DELLEN_sum', 'DEPTH_avg', 'DEPTH_max', 'DEPTH_min', 'DEPTH_std', 'EFFECT_FV_count', 'EFFECT_LOF_ratio', 'EFFECT_NS_count', 'EFFECT_SG_count', 'EFFECT_nunique', 'HB', 'HB_missing', 'ID', 'LEN_avg', 'LEN_max', 'MONOCYTES', 'MONOCYTES_missing', 'Nmut', 'PLT', 'PLT_missing', 'VAF_avg', 'VAF_max', 'VAF_std', 'WBC', 'WBC_missing', 'baseline_chr_count', 'eln_like_flag_adverse_cyto', 'eln_like_flag_intermediate_cyto', 'eln_like_risk_cyto', 'has_abn17p', 'has_any_abnormality', 'has_inv16_or_t_16_16', 'has_inv3_or_t3_3', 'has_minus5_or_del5q', 'has_minus7_or_del7q', 'has_plus8', 'has_t_15_17', 'has_t_6_9', 'has_t_8_21', 'has_t_9_22', 'is_abnormal_karyotype', 'is_complex_karyotype', 'is_cyto_missing_or_failed', '

In [28]:
target_cols = ["OS_STATUS", "OS_YEARS"]

# Train features
X = df_train.drop(columns=target_cols + ["ID"])
X = pd.get_dummies(X, drop_first=True).astype(float)

# Eval features (aligned with train)
X_eval = df_eval.drop(columns=["ID"])
X_eval = pd.get_dummies(X_eval, drop_first=True)
X_eval = X_eval.reindex(columns=X.columns, fill_value=0).astype(float)

# Standardization
scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X),
    columns=X.columns,
    index=X.index,
)
X_eval_scaled = pd.DataFrame(
    scaler.transform(X_eval),
    columns=X_eval.columns,
    index=X_eval.index,
)

X = X_scaled.fillna(0)
X_eval = X_eval_scaled.fillna(0)

# Targets (torch tensors)
y_time = torch.tensor(df_train["OS_YEARS"].values, dtype=torch.float32)
y_event = torch.tensor(df_train["OS_STATUS"].values, dtype=torch.float32)

print(f"X shape      : {X.shape}")
print(f"X_eval shape : {X_eval.shape}")
print(f"NaN in X      : {X.isna().sum().sum()}")
print(f"NaN in X_eval : {X_eval.isna().sum().sum()}")

X shape      : (3173, 234)
X_eval shape : (1193, 234)
NaN in X      : 0
NaN in X_eval : 0


# 2. MTLR Utilities & Risk Scoring

In [29]:
def set_global_seeds(seed: int):
    """Set NumPy and PyTorch seeds for reproducibility."""
    np.random.seed(seed)
    torch.manual_seed(seed)


def build_mtlr_model(input_dim: int, time_bins, params: dict) -> nn.Module:
    """Build an MTLR neural network from a hyperparameter dictionary."""
    if params["activation"] == "relu":
        activation = nn.ReLU()
    elif params["activation"] == "leaky_relu":
        activation = nn.LeakyReLU()
    else:
        activation = nn.ELU()

    model = nn.Sequential(
        nn.Linear(input_dim, params["n_hidden1"]),
        nn.BatchNorm1d(params["n_hidden1"]),
        activation,
        nn.Dropout(params["dropout1"]),
        nn.Linear(params["n_hidden1"], params["n_hidden2"]),
        nn.BatchNorm1d(params["n_hidden2"]),
        activation,
        nn.Dropout(params["dropout2"]),
        MTLR(params["n_hidden2"], len(time_bins)),
    )
    return model


def get_optimizer(model: nn.Module, params: dict):
    """Return an optimizer configured with the given hyperparameters."""
    if params["optimizer"] == "adamw":
        return torch.optim.AdamW(
            model.parameters(),
            lr=params["lr"],
            weight_decay=params["weight_decay"],
        )
    else:
        return torch.optim.Adam(
            model.parameters(),
            lr=params["lr"],
            weight_decay=params["weight_decay"],
        )


def train_mtlr_model(
    model: nn.Module,
    optimizer,
    X_tensor: torch.Tensor,
    y_time_fold: torch.Tensor,
    y_event_fold: torch.Tensor,
    time_bins,
    C1: float,
    n_epochs: int,
    clip_grad: float = 1.0,
    verbose: bool = False,
):
    """
    Train an MTLR model on a given dataset.
    Returns the trained model and the list of losses.
    If NaN occurs in training loss, returns (None, None).
    """
    target = encode_survival(y_time_fold, y_event_fold, time_bins)
    losses = []

    model.train()
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        logits = model(X_tensor)
        loss = mtlr_neg_log_likelihood(
            logits, target, model[-1], C1=C1, average=True
        )

        if torch.isnan(loss):
            # signal failure
            return None, None

        loss.backward()
        if clip_grad is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad)
        optimizer.step()
        losses.append(loss.item())

        if verbose and ((epoch + 1) % max(1, n_epochs // 5) == 0):
            print(f"  Epoch {epoch+1}/{n_epochs} - Loss: {loss.item():.4f}")

    return model, losses


def mtlr_risk_scores(model: nn.Module, X_tensor: torch.Tensor) -> np.ndarray:
    """
    Compute MTLR risk scores using log-sum-exp over logits.
    Higher score = higher risk.
    """
    model.eval()
    with torch.no_grad():
        logits = model(X_tensor)
        risk = torch.logsumexp(logits, dim=1).cpu().numpy()
    return risk


def rank_based_ensemble(risk_matrix: np.ndarray) -> np.ndarray:
    """
    Rank-based ensembling:
      - risk_matrix: shape (n_models, n_samples)
      - For each model, convert risk scores to ranks (1 = lowest risk, N = highest risk),
        then average ranks across models.
    """
    n_models, n_samples = risk_matrix.shape
    ranks_all = np.zeros_like(risk_matrix, dtype=float)

    for m in range(n_models):
        scores = risk_matrix[m]
        order = np.argsort(scores)  # ascending
        ranks = np.empty_like(order, dtype=float)
        ranks[order] = np.arange(1, n_samples + 1)
        ranks_all[m] = ranks

    mean_ranks = ranks_all.mean(axis=0)
    return mean_ranks


def compute_ipcw_cindex(y_train_struct, y_test_struct, risk_scores, tau: float):
    """Compute IPCW C-index on test data."""
    return concordance_index_ipcw(
        y_train_struct,
        y_test_struct,
        risk_scores,
        tau=tau,
    )[0]

# 3. Nested Cross-Validation: Outer (Evaluation) / Inner (Optuna) + 5-Model Ensemble

In [31]:
outer_kf = KFold(n_splits=N_SPLITS_OUTER, shuffle=True, random_state=123)

cv_single_scores = []    # single model = first model in the ensemble
cv_ensemble_scores = []  # ensemble of 5 models (rank-based)

print("=== Starting outer cross-validation ===")

for fold, (train_idx, test_idx) in enumerate(outer_kf.split(X), 1):
    print(f"\n######## Outer Fold {fold}/{N_SPLITS_OUTER} ########")

    # --- Outer fold data ---
    X_train_outer = X.iloc[train_idx].reset_index(drop=True)
    X_test_outer = X.iloc[test_idx].reset_index(drop=True)

    y_time_train_outer = y_time[train_idx]
    y_time_test_outer = y_time[test_idx]
    y_event_train_outer = y_event[train_idx]
    y_event_test_outer = y_event[test_idx]

    X_train_outer_tensor = torch.tensor(
        X_train_outer.values, dtype=torch.float32
    )
    X_test_outer_tensor = torch.tensor(
        X_test_outer.values, dtype=torch.float32
    )

    time_bins_outer = make_time_bins(y_time_train_outer, event=y_event_train_outer)

    # Surv structures for outer C-index
    y_train_struct_outer = Surv.from_arrays(
        event=y_event_train_outer.numpy().astype(bool),
        time=y_time_train_outer.numpy(),
    )
    y_test_struct_outer = Surv.from_arrays(
        event=y_event_test_outer.numpy().astype(bool),
        time=y_time_test_outer.numpy(),
    )

    # --- Ensemble: train 5 independent models on this outer fold ---
    risk_models_test = []
    single_cindex_this_fold = None  # C-index of model 0 (baseline single)

    for m in range(N_MODELS_ENSEMBLE):
        print(f"\n  >> Ensemble model {m+1}/{N_MODELS_ENSEMBLE} (outer fold {fold})")

        # Base seed for this model & fold
        base_seed = 1000 + 100 * fold + m
        set_global_seeds(base_seed)

        # === INNER CV: Hyperparameter optimization on X_train_outer ===
        inner_kf = KFold(
            n_splits=N_SPLITS_INNER, shuffle=True, random_state=base_seed
        )

        def inner_objective(trial):
            # Hyperparameter search space
            params = {
                "n_hidden1": trial.suggest_int("n_hidden1", 32, 256, step=32),
                "n_hidden2": trial.suggest_int("n_hidden2", 16, 128, step=16),
                "dropout1": trial.suggest_float("dropout1", 0.0, 0.5),
                "dropout2": trial.suggest_float("dropout2", 0.0, 0.5),
                "lr": trial.suggest_float("lr", 1e-4, 1e-2, log=True),
                "n_epochs": trial.suggest_int("n_epochs", 50, 200, step=25),
                "C1": trial.suggest_float("C1", 0.1, 5.0, log=True),
                "activation": trial.suggest_categorical(
                    "activation", ["relu", "leaky_relu", "elu"]
                ),
                "optimizer": trial.suggest_categorical(
                    "optimizer", ["adam", "adamw"]
                ),
                "weight_decay": trial.suggest_float(
                    "weight_decay", 1e-6, 1e-3, log=True
                ),
            }

            inner_scores = []

            for inner_fold, (inner_train_idx, inner_val_idx) in enumerate(
                inner_kf.split(X_train_outer), 1
            ):
                set_global_seeds(base_seed + trial.number * 10 + inner_fold)

                X_train_inner = X_train_outer.iloc[inner_train_idx]
                X_val_inner = X_train_outer.iloc[inner_val_idx]

                y_time_train_inner = y_time_train_outer[inner_train_idx]
                y_time_val_inner = y_time_train_outer[inner_val_idx]
                y_event_train_inner = y_event_train_outer[inner_train_idx]
                y_event_val_inner = y_event_train_outer[inner_val_idx]

                X_train_inner_tensor = torch.tensor(
                    X_train_inner.values, dtype=torch.float32
                )
                X_val_inner_tensor = torch.tensor(
                    X_val_inner.values, dtype=torch.float32
                )

                time_bins_inner = make_time_bins(
                    y_time_train_inner, event=y_event_train_inner
                )

                model_inner = build_mtlr_model(
                    input_dim=X_train_inner.shape[1],
                    time_bins=time_bins_inner,
                    params=params,
                )
                optimizer_inner = get_optimizer(model_inner, params)

                model_inner, _ = train_mtlr_model(
                    model=model_inner,
                    optimizer=optimizer_inner,
                    X_tensor=X_train_inner_tensor,
                    y_time_fold=y_time_train_inner,
                    y_event_fold=y_event_train_inner,
                    time_bins=time_bins_inner,
                    C1=params["C1"],
                    n_epochs=params["n_epochs"],
                    clip_grad=1.0,
                    verbose=False,
                )

                if model_inner is None:
                    inner_scores.append(0.5)
                    continue

                y_train_struct_inner = Surv.from_arrays(
                    event=y_event_train_inner.numpy().astype(bool),
                    time=y_time_train_inner.numpy(),
                )
                y_val_struct_inner = Surv.from_arrays(
                    event=y_event_val_inner.numpy().astype(bool),
                    time=y_time_val_inner.numpy(),
                )

                risk_val_inner = mtlr_risk_scores(
                    model_inner, X_val_inner_tensor
                )

                if np.isnan(risk_val_inner).any():
                    inner_scores.append(0.5)
                    continue

                try:
                    cindex_inner = compute_ipcw_cindex(
                        y_train_struct_inner,
                        y_val_struct_inner,
                        risk_val_inner,
                        tau=TAU_CINDEX,
                    )
                except Exception:
                    cindex_inner = 0.5

                inner_scores.append(cindex_inner)

            return float(np.mean(inner_scores))

        # Optuna study for this model & this outer fold
        study = optuna.create_study(
            direction="maximize",
            study_name=f"mtlr_outer_fold{fold}_model{m}",
        )
        study.optimize(
            inner_objective,
            n_trials=N_TRIALS_INNER,
            show_progress_bar=False,
        )

        best_params = study.best_params
        print(f"    - Best inner CV C-index : {study.best_value:.4f}")
        print(f"    - Best params           : {best_params}")

        # === Final training of this model on full outer-train set ===
        model_m = build_mtlr_model(
            input_dim=X_train_outer.shape[1],
            time_bins=time_bins_outer,
            params=best_params,
        )
        optimizer_m = get_optimizer(model_m, best_params)

        model_m, _ = train_mtlr_model(
            model=model_m,
            optimizer=optimizer_m,
            X_tensor=X_train_outer_tensor,
            y_time_fold=y_time_train_outer,
            y_event_fold=y_event_train_outer,
            time_bins=time_bins_outer,
            C1=best_params["C1"],
            n_epochs=best_params["n_epochs"],
            clip_grad=1.0,
            verbose=False,
        )

        if model_m is None:
            print("Training failed on outer train set. Model ignored.")
            continue

        # Risk scores on outer test
        risk_test_m = mtlr_risk_scores(model_m, X_test_outer_tensor)
        risk_models_test.append(risk_test_m)

        # Define "single model" as the first model in the ensemble
        if m == 0:
            if np.isnan(risk_test_m).any():
                cindex_single = 0.5
            else:
                cindex_single = compute_ipcw_cindex(
                    y_train_struct_outer,
                    y_test_struct_outer,
                    risk_test_m,
                    tau=TAU_CINDEX,
                )
            single_cindex_this_fold = cindex_single
            print(f"    -> Single model C-index on outer test : {cindex_single:.4f}")

    # End of the loop over ensemble models for this outer fold

    # If no valid models:
    if len(risk_models_test) == 0:
        print(" No valid models for this fold. Ensemble C-index = 0.5.")
        cindex_ensemble = 0.5
    else:
        risk_matrix_test = np.vstack(risk_models_test)
        risk_ensemble = rank_based_ensemble(risk_matrix_test)
        if np.isnan(risk_ensemble).any():
            cindex_ensemble = 0.5
        else:
            cindex_ensemble = compute_ipcw_cindex(
                y_train_struct_outer,
                y_test_struct_outer,
                risk_ensemble,
                tau=TAU_CINDEX,
            )

    # If single model did not compute (e.g. model 0 failed)
    if single_cindex_this_fold is None:
        single_cindex_this_fold = 0.5

    cv_single_scores.append(single_cindex_this_fold)
    cv_ensemble_scores.append(cindex_ensemble)

    print(f"\n  Fold {fold} summary:")
    print(f"    Single model C-index : {single_cindex_this_fold:.4f}")
    print(f"    Ensemble C-index     : {cindex_ensemble:.4f}")

[I 2025-12-30 03:48:16,809] A new study created in memory with name: mtlr_outer_fold1_model0


=== Starting outer cross-validation ===

######## Outer Fold 1/5 ########

  >> Ensemble model 1/5 (outer fold 1)


[I 2025-12-30 03:48:23,342] Trial 0 finished with value: 0.6910235644081962 and parameters: {'n_hidden1': 192, 'n_hidden2': 48, 'dropout1': 0.12408085911158667, 'dropout2': 0.47628999431304947, 'lr': 0.00010402224345093958, 'n_epochs': 175, 'C1': 0.22225575749462878, 'activation': 'relu', 'optimizer': 'adamw', 'weight_decay': 1.2398352579764662e-05}. Best is trial 0 with value: 0.6910235644081962.
[I 2025-12-30 03:48:27,873] Trial 1 finished with value: 0.6875893803320823 and parameters: {'n_hidden1': 64, 'n_hidden2': 112, 'dropout1': 0.2386563824108725, 'dropout2': 0.12941603950046587, 'lr': 0.0006693781314791485, 'n_epochs': 150, 'C1': 3.9135337378661674, 'activation': 'relu', 'optimizer': 'adam', 'weight_decay': 9.368012874082294e-05}. Best is trial 0 with value: 0.6910235644081962.
[I 2025-12-30 03:48:32,320] Trial 2 finished with value: 0.654337487798506 and parameters: {'n_hidden1': 128, 'n_hidden2': 80, 'dropout1': 0.1703787566878739, 'dropout2': 0.18689513631393645, 'lr': 0.009

    - Best inner CV C-index : 0.6973
    - Best params           : {'n_hidden1': 256, 'n_hidden2': 48, 'dropout1': 0.38606644660561784, 'dropout2': 0.04118874849851778, 'lr': 0.0002161533672489277, 'n_epochs': 50, 'C1': 1.2972850734225643, 'activation': 'elu', 'optimizer': 'adamw', 'weight_decay': 0.0009858901702705267}


[I 2025-12-30 03:50:04,327] A new study created in memory with name: mtlr_outer_fold1_model1


...

    - Best inner CV C-index : 0.7031
    - Best params           : {'n_hidden1': 32, 'n_hidden2': 32, 'dropout1': 0.4509300178174399, 'dropout2': 0.444713293751371, 'lr': 0.0004175449789275625, 'n_epochs': 200, 'C1': 0.6332390305408805, 'activation': 'leaky_relu', 'optimizer': 'adam', 'weight_decay': 0.000483463766425957}


[I 2025-12-30 04:47:04,490] A new study created in memory with name: mtlr_outer_fold5_model4



  >> Ensemble model 5/5 (outer fold 5)


[I 2025-12-30 04:47:07,996] Trial 0 finished with value: 0.6844167007528741 and parameters: {'n_hidden1': 96, 'n_hidden2': 128, 'dropout1': 0.1547380642658553, 'dropout2': 0.3351403000983689, 'lr': 0.00019234283353601514, 'n_epochs': 75, 'C1': 1.1789307759178338, 'activation': 'relu', 'optimizer': 'adam', 'weight_decay': 0.00012088280120260011}. Best is trial 0 with value: 0.6844167007528741.
[I 2025-12-30 04:47:17,559] Trial 1 finished with value: 0.6240188397705668 and parameters: {'n_hidden1': 256, 'n_hidden2': 112, 'dropout1': 0.12006938644454251, 'dropout2': 0.09208058984661444, 'lr': 0.004099447867261757, 'n_epochs': 150, 'C1': 1.3916651059689598, 'activation': 'leaky_relu', 'optimizer': 'adam', 'weight_decay': 2.914310970772293e-06}. Best is trial 0 with value: 0.6844167007528741.
[I 2025-12-30 04:47:21,624] Trial 2 finished with value: 0.6837650898830079 and parameters: {'n_hidden1': 160, 'n_hidden2': 96, 'dropout1': 0.400614440719313, 'dropout2': 0.16984035466398456, 'lr': 0.0

    - Best inner CV C-index : 0.6981
    - Best params           : {'n_hidden1': 32, 'n_hidden2': 32, 'dropout1': 0.44356256542954703, 'dropout2': 0.03308675013854269, 'lr': 0.001496265848259566, 'n_epochs': 125, 'C1': 3.325564035792218, 'activation': 'relu', 'optimizer': 'adamw', 'weight_decay': 0.0009848321041793298}

  Fold 5 summary:
    Single model C-index : 0.6843
    Ensemble C-index     : 0.6989


# 4. Statistical Analysis: Means, 95% CI, Paired t-test

In [40]:
def summarize_scores(scores, name: str):
    scores = np.array(scores)
    mean = scores.mean()
    std = scores.std(ddof=1)
    n = len(scores)

    t_crit = stats.t.ppf(0.975, df=n - 1)
    ci_low = mean - t_crit * std / np.sqrt(n)
    ci_high = mean + t_crit * std / np.sqrt(n)

    print(f"\n{name}")
    print(f"  Scores by fold : {np.round(scores, 4)}")
    print(f"  Mean           : {mean:.4f}")
    print(f"  Std dev        : {std:.4f}")
    print(f"  95% CI         : [{ci_low:.4f} ; {ci_high:.4f}]")
    return scores, mean, std, (ci_low, ci_high)


single_scores, single_mean, single_std, single_ci = summarize_scores(
    cv_single_scores, "Single model (first model of ensemble)"
)
ensemble_scores, ensemble_mean, ensemble_std, ensemble_ci = summarize_scores(
    cv_ensemble_scores, "Ensemble (5 models, rank-based)"
)

# Paired t-test: ensemble vs single
diff = ensemble_scores - single_scores
t_stat, p_val = stats.ttest_rel(ensemble_scores, single_scores)

print("\nPaired t-test: Ensemble vs Single model")
print(f"  Fold-wise differences (ensemble - single) : {np.round(diff, 4)}")
print(f"  Mean difference : {diff.mean():.4f}")
print(f"  t-statistic     : {t_stat:.4f}")
print(f"  p-value         : {p_val:.4f}")
if p_val < 0.05:
    print("  => Statistically significant improvement at 5% level")
else:
    print("  => NOT statistically significant at 5% level with these folds")

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Visualization with Plotly
folds = np.arange(1, N_SPLITS_OUTER + 1)

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("C-index per outer fold", "Average performance (±1 SD)")
)

# ---- Left subplot: fold-wise scores ----
fig.add_trace(
    go.Scatter(
        x=folds,
        y=single_scores,
        mode="lines+markers",
        name="Single model",
        marker=dict(symbol="circle", size=10),
        line=dict(width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=folds,
        y=ensemble_scores,
        mode="lines+markers",
        name="Ensemble (rank)",
        marker=dict(symbol="square", size=10),
        line=dict(width=2)
    ),
    row=1, col=1
)

fig.update_xaxes(title_text="Outer fold", row=1, col=1)
fig.update_yaxes(title_text="IPCW C-index", row=1, col=1)

# ---- Right subplot: bar plot with std ----
fig.add_trace(
    go.Bar(
        x=["Single", "Ensemble"],
        y=[single_mean, ensemble_mean],
        error_y=dict(
            type="data",
            array=[single_std, ensemble_std],
            visible=True
        ),
        name="Mean ± 1 SD"
    ),
    row=1, col=2
)

fig.update_yaxes(title_text="IPCW C-index (mean)", row=1, col=2)

fig.update_layout(
    width=900,
    height=400,
    showlegend=True,
    title_text="Model comparison: single vs ensemble",
    template="plotly_white",              # <-- enlève le fond gris
    paper_bgcolor="white",                # <-- (optionnel) fond global blanc
    plot_bgcolor="white"                  # <-- (optionnel) fond des axes blanc
)


fig.show()



Single model (first model of ensemble)
  Scores by fold : [0.714  0.7137 0.7061 0.6803 0.6843]
  Mean           : 0.6997
  Std dev        : 0.0163
  95% CI         : [0.6795 ; 0.7199]

Ensemble (5 models, rank-based)
  Scores by fold : [0.7191 0.7248 0.7185 0.6895 0.6989]
  Mean           : 0.7102
  Std dev        : 0.0151
  95% CI         : [0.6914 ; 0.7290]

Paired t-test: Ensemble vs Single model
  Fold-wise differences (ensemble - single) : [0.0051 0.0111 0.0123 0.0093 0.0145]
  Mean difference : 0.0105
  t-statistic     : 6.5553
  p-value         : 0.0028
  => Statistically significant improvement at 5% level


# 5. Final Training on Full Train Set + 5-Model Ensemble for Evaluation Predictions

In [33]:
def optimize_and_train_full_model(X_data, y_time_data, y_event_data, base_seed: int):
    """
    On the full dataset:
      - Inner CV to optimize hyperparameters (with Optuna)
      - Final training on the full data with best hyperparameters
    Returns the trained model, time bins, best params, and loss history.
    """
    set_global_seeds(base_seed)

    X_data = X_data.reset_index(drop=True)
    X_tensor = torch.tensor(X_data.values, dtype=torch.float32)
    time_bins_data = make_time_bins(y_time_data, event=y_event_data)

    inner_kf = KFold(
        n_splits=N_SPLITS_INNER, shuffle=True, random_state=base_seed
    )

    def inner_objective_full(trial):
        params = {
            "n_hidden1": trial.suggest_int("n_hidden1", 32, 256, step=32),
            "n_hidden2": trial.suggest_int("n_hidden2", 16, 128, step=16),
            "dropout1": trial.suggest_float("dropout1", 0.0, 0.5),
            "dropout2": trial.suggest_float("dropout2", 0.0, 0.5),
            "lr": trial.suggest_float("lr", 1e-4, 1e-2, log=True),
            "n_epochs": trial.suggest_int("n_epochs", 50, 200, step=25),
            "C1": trial.suggest_float("C1", 0.1, 5.0, log=True),
            "activation": trial.suggest_categorical(
                "activation", ["relu", "leaky_relu", "elu"]
            ),
            "optimizer": trial.suggest_categorical(
                "optimizer", ["adam", "adamw"]
            ),
            "weight_decay": trial.suggest_float(
                "weight_decay", 1e-6, 1e-3, log=True
            ),
        }

        inner_scores = []

        for inner_fold, (inner_train_idx, inner_val_idx) in enumerate(
            inner_kf.split(X_data), 1
        ):
            set_global_seeds(base_seed + trial.number * 10 + inner_fold)

            X_train_inner = X_data.iloc[inner_train_idx]
            X_val_inner = X_data.iloc[inner_val_idx]

            y_time_train_inner = y_time_data[inner_train_idx]
            y_time_val_inner = y_time_data[inner_val_idx]
            y_event_train_inner = y_event_data[inner_train_idx]
            y_event_val_inner = y_event_data[inner_val_idx]

            X_train_inner_tensor = torch.tensor(
                X_train_inner.values, dtype=torch.float32
            )
            X_val_inner_tensor = torch.tensor(
                X_val_inner.values, dtype=torch.float32
            )

            time_bins_inner = make_time_bins(
                y_time_train_inner, event=y_event_train_inner
            )

            model_inner = build_mtlr_model(
                input_dim=X_train_inner.shape[1],
                time_bins=time_bins_inner,
                params=params,
            )
            optimizer_inner = get_optimizer(model_inner, params)

            model_inner, _ = train_mtlr_model(
                model=model_inner,
                optimizer=optimizer_inner,
                X_tensor=X_train_inner_tensor,
                y_time_fold=y_time_train_inner,
                y_event_fold=y_event_train_inner,
                time_bins=time_bins_inner,
                C1=params["C1"],
                n_epochs=params["n_epochs"],
                clip_grad=1.0,
                verbose=False,
            )

            if model_inner is None:
                inner_scores.append(0.5)
                continue

            y_train_struct_inner = Surv.from_arrays(
                event=y_event_train_inner.numpy().astype(bool),
                time=y_time_train_inner.numpy(),
            )
            y_val_struct_inner = Surv.from_arrays(
                event=y_event_val_inner.numpy().astype(bool),
                time=y_time_val_inner.numpy(),
            )

            risk_val_inner = mtlr_risk_scores(model_inner, X_val_inner_tensor)

            if np.isnan(risk_val_inner).any():
                inner_scores.append(0.5)
                continue

            try:
                cindex_inner = compute_ipcw_cindex(
                    y_train_struct_inner,
                    y_val_struct_inner,
                    risk_val_inner,
                    tau=TAU_CINDEX,
                )
            except Exception:
                cindex_inner = 0.5

            inner_scores.append(cindex_inner)

        return float(np.mean(inner_scores))

    # Optuna on full dataset (inner CV)
    study_full = optuna.create_study(
        direction="maximize",
        study_name=f"mtlr_full_seed{base_seed}",
    )
    study_full.optimize(
        inner_objective_full,
        n_trials=N_TRIALS_INNER,
        show_progress_bar=False,
    )

    best_params_full = study_full.best_params
    print(f"Seed {base_seed} - best inner CV C-index: {study_full.best_value:.4f}")
    print(f"Best params: {best_params_full}")

    # Final training on full data
    model_full = build_mtlr_model(
        input_dim=X_data.shape[1],
        time_bins=time_bins_data,
        params=best_params_full,
    )
    optimizer_full = get_optimizer(model_full, best_params_full)

    model_full, losses_full = train_mtlr_model(
        model=model_full,
        optimizer=optimizer_full,
        X_tensor=X_tensor,
        y_time_fold=y_time_data,
        y_event_fold=y_event_data,
        time_bins=time_bins_data,
        C1=best_params_full["C1"],
        n_epochs=best_params_full["n_epochs"],
        clip_grad=1.0,
        verbose=True,
    )

    return model_full, time_bins_data, best_params_full, losses_full

In [34]:
# Train 5 final models on the full train set
X_tensor_full = torch.tensor(X.values, dtype=torch.float32)
X_eval_tensor = torch.tensor(X_eval.values, dtype=torch.float32)

models_final = []
risk_matrix_eval = []

print("\n=== Final ensemble training (5 models) on the full train set ===")

for m in range(N_MODELS_ENSEMBLE):
    base_seed = 5000 + m
    print(f"\n##### Final model {m+1}/{N_MODELS_ENSEMBLE} (seed={base_seed}) #####")
    model_m, time_bins_full, best_params_m, losses_m = optimize_and_train_full_model(
        X_data=X, y_time_data=y_time, y_event_data=y_event, base_seed=base_seed
    )

    if model_m is None:
        print("⚠ Final model is invalid, ignored.")
        continue

    models_final.append(model_m)
    risk_eval_m = mtlr_risk_scores(model_m, X_eval_tensor)
    risk_matrix_eval.append(risk_eval_m)

[I 2025-12-30 04:49:30,090] A new study created in memory with name: mtlr_full_seed5000



=== Final ensemble training (5 models) on the full train set ===

##### Final model 1/5 (seed=5000) #####


[I 2025-12-30 04:49:40,596] Trial 0 finished with value: 0.672540259934299 and parameters: {'n_hidden1': 160, 'n_hidden2': 112, 'dropout1': 0.18074339884316265, 'dropout2': 0.13619192326714752, 'lr': 0.00442273665163735, 'n_epochs': 125, 'C1': 1.212386256790836, 'activation': 'elu', 'optimizer': 'adam', 'weight_decay': 5.910581614754514e-06}. Best is trial 0 with value: 0.672540259934299.
[I 2025-12-30 04:49:49,159] Trial 1 finished with value: 0.6854192496573531 and parameters: {'n_hidden1': 224, 'n_hidden2': 80, 'dropout1': 0.1392381710106912, 'dropout2': 0.4616361539738322, 'lr': 0.00010443252426056724, 'n_epochs': 100, 'C1': 2.532564453579163, 'activation': 'leaky_relu', 'optimizer': 'adam', 'weight_decay': 5.4750838929219234e-05}. Best is trial 1 with value: 0.6854192496573531.
[I 2025-12-30 04:50:01,336] Trial 2 finished with value: 0.6560306423851398 and parameters: {'n_hidden1': 224, 'n_hidden2': 64, 'dropout1': 0.2175738566484522, 'dropout2': 0.49284353115388185, 'lr': 0.00939

Seed 5000 - best inner CV C-index: 0.7022
Best params: {'n_hidden1': 64, 'n_hidden2': 16, 'dropout1': 0.4407958755594277, 'dropout2': 0.0940503748881036, 'lr': 0.0006190024535060475, 'n_epochs': 75, 'C1': 0.253979105483526, 'activation': 'relu', 'optimizer': 'adam', 'weight_decay': 1.1599514321504961e-05}
  Epoch 15/75 - Loss: 5.0309
  Epoch 30/75 - Loss: 4.6518
  Epoch 45/75 - Loss: 4.3704
  Epoch 60/75 - Loss: 4.1179


[I 2025-12-30 04:52:47,428] A new study created in memory with name: mtlr_full_seed5001


...

Seed 5003 - best inner CV C-index: 0.6965
Best params: {'n_hidden1': 32, 'n_hidden2': 64, 'dropout1': 0.20423248158715018, 'dropout2': 0.4307059192784426, 'lr': 0.0009483405605141187, 'n_epochs': 125, 'C1': 1.7093133152819933, 'activation': 'relu', 'optimizer': 'adamw', 'weight_decay': 0.0003227708241974583}
  Epoch 25/125 - Loss: 36.9157
  Epoch 50/125 - Loss: 27.9553
  Epoch 75/125 - Loss: 20.9522
  Epoch 100/125 - Loss: 15.6019


[I 2025-12-30 05:04:42,522] A new study created in memory with name: mtlr_full_seed5004


  Epoch 125/125 - Loss: 11.4300

##### Final model 5/5 (seed=5004) #####


[I 2025-12-30 05:04:51,123] Trial 0 finished with value: 0.6895356311841893 and parameters: {'n_hidden1': 32, 'n_hidden2': 96, 'dropout1': 0.3851594377525964, 'dropout2': 0.3639199644393628, 'lr': 0.0007462450567928833, 'n_epochs': 150, 'C1': 0.1716428165866354, 'activation': 'elu', 'optimizer': 'adamw', 'weight_decay': 9.785740101019697e-05}. Best is trial 0 with value: 0.6895356311841893.
[I 2025-12-30 05:04:57,834] Trial 1 finished with value: 0.687057235181746 and parameters: {'n_hidden1': 160, 'n_hidden2': 96, 'dropout1': 0.17630360768796627, 'dropout2': 0.26826520711909413, 'lr': 0.00019104807329267475, 'n_epochs': 75, 'C1': 2.209606974565312, 'activation': 'elu', 'optimizer': 'adam', 'weight_decay': 0.00047553689379499535}. Best is trial 0 with value: 0.6895356311841893.
[I 2025-12-30 05:05:11,231] Trial 2 finished with value: 0.6915508933745974 and parameters: {'n_hidden1': 160, 'n_hidden2': 64, 'dropout1': 0.45982042985556415, 'dropout2': 0.10129358015591072, 'lr': 0.000137023

Seed 5004 - best inner CV C-index: 0.7058
Best params: {'n_hidden1': 64, 'n_hidden2': 112, 'dropout1': 0.48768325923843253, 'dropout2': 0.42322897258181097, 'lr': 0.0008537983624200127, 'n_epochs': 75, 'C1': 3.6998608281196903, 'activation': 'leaky_relu', 'optimizer': 'adamw', 'weight_decay': 2.180802978122992e-06}
  Epoch 15/75 - Loss: 92.1085
  Epoch 30/75 - Loss: 76.0332
  Epoch 45/75 - Loss: 62.5018
  Epoch 60/75 - Loss: 51.1331
  Epoch 75/75 - Loss: 41.5606


In [35]:
risk_matrix_eval = np.array(risk_matrix_eval)
print(f"\nNumber of valid final models in ensemble: {risk_matrix_eval.shape[0]}")

if risk_matrix_eval.shape[0] == 0:
    raise RuntimeError("No valid final model for the ensemble.")

# Final ensemble: rank-based aggregation
risk_ensemble_eval = rank_based_ensemble(risk_matrix_eval)

# Normalization to [0, 1] for interpretability
risk_min = risk_ensemble_eval.min()
risk_max = risk_ensemble_eval.max()
risk_ensemble_eval_norm = (risk_ensemble_eval - risk_min) / (risk_max - risk_min + 1e-8)

print(f"Min risk (raw ensemble) : {risk_min:.4f}")
print(f"Max risk (raw ensemble) : {risk_max:.4f}")


Number of valid final models in ensemble: 5
Min risk (raw ensemble) : 7.6000
Max risk (raw ensemble) : 1191.2000


In [41]:
# Build prediction dataframe
submission = pd.DataFrame({
    "ID": df_eval["ID"],
    "risk_score": risk_ensemble_eval_norm,  # or risk_ensemble_eval if you prefer raw scores
})

print(submission.head())

submission.to_csv("../../submissions/submission_mtlr_ensemble_nestedcv.csv", index=False)
print("\nPrediction file saved as submission_mtlr_ensemble_nestedcv.csv")

     ID  risk_score
0  KYW1    0.891348
1  KYW2    0.757857
2  KYW3    0.496452
3  KYW4    0.728625
4  KYW5    0.781176

Prediction file saved as submission_mtlr_ensemble_nestedcv.csv
